<a href="https://colab.research.google.com/github/adarsh-nl/RLHF-based-ImgaeCaptionGeneration/blob/main/Feedback_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Concatenate
from sklearn.model_selection import train_test_split
import pickle

# Load the data
data = pd.read_csv('/content/main_preference_dataset.csv')

# Define the maximum caption length and vocabulary size
max_len = max([len(caption.split()) for caption in data['caption1']])
max_words = 10000

# Tokenize the captions
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['caption1'])
captions_tokenized = tokenizer.texts_to_sequences(data['caption1'])
captions_padded = pad_sequences(captions_tokenized, padding='post', maxlen=max_len)

# Convert the image features to arrays
image_features = np.array([np.fromstring(x[1:-1], sep=',') for x in data['image']])

# Split the data into train, val, and test sets
train_captions, test_captions, train_images, test_images, train_labels, test_labels = train_test_split(
    captions_padded, image_features, data['feedback1'], test_size=0.2, random_state=42)

train_captions, val_captions, train_images, val_images, train_labels, val_labels = train_test_split(
    train_captions, train_images, train_labels, test_size=0.2, random_state=42)

# Define the model architecture
caption_input = Input(shape=(max_len,))
caption_embedded = Embedding(max_words, 100)(caption_input)
caption_lstm = LSTM(100, dropout=0.5)(caption_embedded)
image_input = Input(shape=(image_features.shape[1],))
combined = Concatenate()([caption_lstm, image_input])
dense1 = Dense(100, activation='relu')(combined)
dropout1 = Dropout(0.5)(dense1)
output = Dense(1, activation='sigmoid')(dropout1)
model = Model(inputs=[caption_input, image_input], outputs=output)

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit([train_captions, train_images], train_labels, epochs=10, batch_size=32,
          validation_data=([val_captions, val_images], val_labels))

# Save the model and tokenizer
model.save('my_model.h5')
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Evaluate the model on the test set
loss, accuracy = model.evaluate([test_captions, test_images], test_labels)
print('Test loss:', loss)
print('Test accuracy:', accuracy)


<ipython-input-89-d7808e655745>:25: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  image_features = np.array([np.fromstring(x[1:-1], sep=',') for x in data['image']])


Epoch 1/10
5/5 [==============================] - 4s 295ms/step - loss: 0.2181 - accuracy: 0.0000e+00 - val_loss: 0.1491 - val_accuracy: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 0s 74ms/step - loss: 0.0979 - accuracy: 0.0000e+00 - val_loss: 0.0078 - val_accuracy: 0.0000e+00
Epoch 3/10
5/5 [==============================] - 0s 77ms/step - loss: 0.0040 - accuracy: 0.0000e+00 - val_loss: 3.5100e-07 - val_accuracy: 0.0000e+00
Epoch 4/10
5/5 [==============================] - 0s 80ms/step - loss: 2.6900e-06 - accuracy: 0.0000e+00 - val_loss: 6.0286e-08 - val_accuracy: 0.0000e+00
Epoch 5/10
5/5 [==============================] - 0s 80ms/step - loss: 2.1947e-06 - accuracy: 0.0000e+00 - val_loss: 6.4240e-08 - val_accuracy: 0.0000e+00
Epoch 6/10
5/5 [==============================] - 0s 89ms/step - loss: 1.7949e-06 - accuracy: 0.0000e+00 - val_loss: 6.5396e-08 - val_accuracy: 0.0000e+00
Epoch 7/10
5/5 [==============================] - 0s 73ms/step - loss: 1.7640e-06 - accur